In [1]:
import os
import random
import numpy as np

def set_seed(seed: int):
    random.seed(seed) # Python
    np.random.seed(seed)  # Numpy, é o gerador utilizado pelo sklearn
    os.environ["PYTHONHASHSEED"] = str(seed)  # sistema operativo

set_seed(25)

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)        # No max width

In [3]:
df = pd.read_csv("data/csvs/flow_data_clean_v2.csv")

In [4]:
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0,0,1,0,82,0,82,82,82.0,0.0,0,9223372036854775807,0.0,0.0,0.000000e+00,0.000000,0.000000,0.0,0,9223372036854775807,0,0.0,0.0,0,9223372036854775807,0,0.0,0.0,0,9223372036854775807,0,0,0,0,20,0,0.000000,0.000000,82,82,82.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,82.0,82.0,0.0,0,0,0,0,0,0,1,82,0,0,0,0,1,82,0,0,0,0,0,0,0,0
1,0,0,1,0,82,0,82,82,82.0,0.0,0,9223372036854775807,0.0,0.0,0.000000e+00,0.000000,0.000000,0.0,0,9223372036854775807,0,0.0,0.0,0,9223372036854775807,0,0.0,0.0,0,9223372036854775807,0,0,0,0,20,0,0.000000,0.000000,82,82,82.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,82.0,82.0,0.0,0,0,0,0,0,0,1,82,0,0,0,0,1,82,0,0,0,0,0,0,0,0
2,0,0,1,0,82,0,82,82,82.0,0.0,0,9223372036854775807,0.0,0.0,0.000000e+00,0.000000,0.000000,0.0,0,9223372036854775807,0,0.0,0.0,0,9223372036854775807,0,0.0,0.0,0,9223372036854775807,0,0,0,0,20,0,0.000000,0.000000,82,82,82.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,82.0,82.0,0.0,0,0,0,0,0,0,1,82,0,0,0,0,1,82,0,0,0,0,0,0,0,0
3,48780,0,1,1,278,66,278,278,278.0,0.0,66,66,66.0,0.0,1.534937e+07,89240.510638,0.000022,0.0,0,0,0,0.0,0.0,0,9223372036854775807,0,0.0,0.0,0,9223372036854775807,1,0,0,0,264,52,44620.255319,44620.255319,66,278,172.0,0.0,0.0,0,0,0,1,2,0,0,0,1.0,172.0,278.0,66.0,0,0,0,0,0,0,1,278,1,66,501,627,1,278,0,0,0,0,0,0,0,0
4,48794,0,1,1,275,66,275,275,275.0,0.0,66,66,66.0,0.0,1.288520e+07,75573.045045,0.000026,0.0,0,0,0,0.0,0.0,0,9223372036854775807,0,0.0,0.0,0,9223372036854775807,1,0,0,0,261,52,37786.522523,37786.522523,66,275,170.5,0.0,0.0,0,0,0,1,2,0,0,0,1.0,170.5,275.0,66.0,0,0,0,0,0,0,1,275,1,66,501,627,1,275,0,0,0,0,0,0,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10249 entries, 0 to 10248
Data columns (total 77 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Destination Port             10249 non-null  int64  
 1   Flow Duration                10249 non-null  int64  
 2   Total Fwd Packets            10249 non-null  int64  
 3   Total Backward Packets       10249 non-null  int64  
 4   Total Length of Fwd Packets  10249 non-null  int64  
 5   Total Length of Bwd Packets  10249 non-null  int64  
 6   Fwd Packet Length Max        10249 non-null  int64  
 7   Fwd Packet Length Min        10249 non-null  int64  
 8   Fwd Packet Length Mean       10249 non-null  float64
 9   Fwd Packet Length Std        10249 non-null  float64
 10  Bwd Packet Length Max        10249 non-null  int64  
 11  Bwd Packet Length Min        10249 non-null  int64  
 12  Bwd Packet Length Mean       10249 non-null  float64
 13  Bwd Packet Lengt

### Pre Proc

In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Separar features (todas as colunas)
X = df.values

# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### Isolation Forest

In [7]:
from sklearn.ensemble import IsolationForest

# Criar modelo
iso_forest = IsolationForest(contamination=0.01, random_state=42)
iso_forest.fit(X_scaled)

# Fazer previsões
y_pred_iso = iso_forest.predict(X_scaled)

# Resultado: -1 = anomalia, 1 = normal
anomalias_iso = (y_pred_iso == -1).sum()
print(f"Anomalias encontradas pelo Isolation Forest: {anomalias_iso}")


Anomalias encontradas pelo Isolation Forest: 102


### One Classe SVM

In [8]:
from sklearn.svm import OneClassSVM

# Criar modelo
oc_svm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.01)  # nu é a taxa esperada de anomalias
oc_svm.fit(X_scaled)

# Previsões
y_pred_svm = oc_svm.predict(X_scaled)

anomalias_svm = (y_pred_svm == -1).sum()
print(f"Anomalias encontradas pelo One-Class SVM: {anomalias_svm}")


Anomalias encontradas pelo One-Class SVM: 246


### Local Outlier Factor

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

# Criar e prever diretamente
lof = LocalOutlierFactor(n_neighbors=1000, contamination="auto")
y_pred_lof = lof.fit_predict(X_scaled)

anomalias_lof = (y_pred_lof == -1).sum()
print(f"Anomalias encontradas pelo LOF: {anomalias_lof}")


Anomalias encontradas pelo LOF: 10


/home/eduardo/miniconda3/envs/env/lib/python3.10/site-packages/sklearn/neighbors/_lof.py:322: UserWarning: Duplicate values are leading to incorrect results. Increase the number of neighbors for more accurate results.
  warnings.warn(
